# GEM Tutorial with COBRApy

This exercise will walk through some of the basic operations in working with a genome-scale metabolic model (GEM). The vast majority of software that has been developed surrounding GEMs has been done in MATLAB, likely because this form of modeling has origins in engineering (specifically chemical engineering). Although well-suited for metabolic modeling, MATLAB is not open-source and therefore limits the accessibility of such software. Fortunately, the modeling community has implemented the MATLAB COnstrant-Based Reconstruction and Analysis [(COBRA) Toolbox](https://opencobra.github.io/cobratoolbox/stable/) in Python, as [**COBRApy**](https://opencobra.github.io/cobrapy/).

**COBRApy** is still relatively new and therefore lacks some of the functionality of its MATLAB counterparts, but the core utilities are available and quickly expanding. Here, we will demonstrate some of the basic functions and classes of the **COBRApy** package, which should also familiarize the user with the fundamentals of GEM structure and simulation.

Most of the commands and material covered in this tutorial can be found in the [**COBRApy Documentation**](https://cobrapy.readthedocs.io/en/stable/), so we encourage you to reference the documentation if you encounter errors, warnings, or need further detail about something. You can of course always ask us for help too :)

In [ ]:
import cobra
import cobra.test
import os

## View the global configuration object

Before jumping right into things, it is always nice to see what sort of default settings are in place. **COBRApy** has organized such defaults into a **global configuration object**, which can be viewed or adjusted as needed.

In [ ]:
cobra_config = cobra.Configuration()

In [ ]:
# view a brief summary of the object
cobra_config

In [ ]:
# view the default reaction flux bounds (min, max)
cobra_config.bounds

## Import and inspect a small test model

GEMs, as their name implies ("_genome_"-scale), are often quite large, containing thousands of reactions, metabolites, and genes. It is therefore best to begin working with a simplified model that is quick to load and use, and easy to conceptualize.

For this exercise, we will use the `textbook` model that is provided with the **COBRApy** package. This model encompasses the core pathways of central carbon metabolism in the _E. coli_ bacterium.

In [ ]:
# the cobra package ships with several test models in different formats
data_dir = cobra.test.data_dir
os.listdir(data_dir)[:10]

In [ ]:
# load the "textbook" model from the SBML (.xml) file
model = cobra.io.read_sbml_model(os.path.join(data_dir, "textbook.xml.gz"))
model

**Note:** SBML ([Systems Biology Markup Language](http://sbml.org/Main_Page)) is an XML-based format commonly used to store GEMs. The aim of SBML is to serve as an open and standardized format to facilitate sharing of models and software. Feel free to open the `textbook.xml` file in a text editor to get an idea of how it is formatted.

In [ ]:
# list the first few reactions in the model, with their reaction equations
for x in model.reactions[:10]:
    print("%s : %s" % (x.id, x.reaction))

In [ ]:
# inspect a reaction (e.g., AKGDH) in more detail
model.reactions.AKGDH

Here we can see some of the reaction-associated information that is stored in the model. The `Reaction identifier` must be a unique string, and is typically a short abbreviation or code since there is also a more descriptive `Name` field.

**Question:** Is this reaction irreversible or reversible? From what field(s) can this be determined?

**Question:** Based on the gene-protein rule (GPR), is the reaction catalyzed by *isozymes* or an *enzyme complex*?

In [ ]:
# list the first few metabolites in the model, along with their chemical formula
for x in model.metabolites[:10]:
    print("%s : %s" % (x.id, x.formula))

In [ ]:
# inspect the 3pg_e metabolite in more detail
model.metabolites.get_by_id('3pg_c')

Note that we cannot reference the metabolite using `model.metabolites.3pg_c` because the metabolite ID begins with a number, which python doesn't like.

We can see the abbreviation of the compartment in which the metabolite exists, though `c` by itself is not super informative.

**Question:** What is the name of the `c` compartment, and what other compartments does the model have?

## Add a new reaction to the model

GEMs are never really "finished" because we continue to find errors or missing content, new reactions/genes/metabolites are discovered, etc. It is therefore common for a user to need to add or remove content from the GEM.

For this example, we will add the [aspartate aminotransferase reaction](https://ecocyc.org/ECOLI/NEW-IMAGE?type=REACTION&object=ASPAMINOTRANS-RXN) to enable the synthesis of aspartate:

`L-glutamate + oxaloacetate <==> 2-oxoglutarate + L-aspartate`

### Create and edit the reaction object

In [ ]:
# create a template reaction and determine what information we need to provide
reaction = cobra.Reaction('ASPAMTR')
reaction

In [ ]:
# add the reaction name
reaction.name = 'aspartate aminotransferase'

In [ ]:
# we need to find the IDs of the metabolites involved in the reaction
met_patterns = ['glutamate', 'oxaloacetate', 'oxoglutarate', 'aspartate']

for met in model.metabolites:
    if any([x in met.name.lower() for x in met_patterns]):
        print("%s : %s" % (met.id, met.name))

Two interesting observations:
1. There are two instances of `2-Oxoglutarate` and `L-Glutamate`
2. Aspartate is not yet in the model

For the first point, note that the `_c` and `_e` suffixes represent the compartment to which the metabolite belongs. Note that not all GEMs append the compartment information to the metabolite IDs, but it is quite common.

In [ ]:
# view model compartments
model.compartments

We want to add our reaction to the cytosol (`c`) compartment, so we will use the `_c` form of the metabolites.

For the second point, we will need to add aspartate to the model.

### Create a new metabolite object

In [ ]:
# create the aspartate metabolite
asp_c = cobra.Metabolite('asp_c')
asp_c  # view its (missing) properties

In [ ]:
# fill in some information about the new aspartate metabolite
asp_c.name = 'L-Aspartate'
asp_c.formula = 'C4H6NO4'
asp_c.compartment='c'

In [ ]:
# now we can add the metabolites and their stoichiometric coefficients to the new reaction
reaction.add_metabolites({
    model.metabolites.glu__L_c: -1.0,
    model.metabolites.oaa_c: -1.0,
    model.metabolites.akg_c: 1.0,
    asp_c: 1.0
})

In [ ]:
# view the reaction details to verify that everything looks correct
reaction

In [ ]:
# view the reversibility of the reaction (should be reversible)
reaction.reversibility

In [ ]:
reaction.reversibility = True  # note that we cannot directly edit the "reversibility" field

In [ ]:
# instead we need to change the lower bound of the reaction
reaction.lower_bound = -1000
reaction.reversibility  # verify that the reversibilty has been updated

In [ ]:
# note that the equation now shows the double-sided arrow "<=>" instead of "-->"
reaction

### Add a gene-protein-reaction (GPR) rule to the reaction

The reaction will still function even if it doesn't have a GPR (GEMs contain many non-enzymatic reactions, after all), but this information is important to include since it can affect some analyses, such as gene deletion analysis or reporter metabolite analysis.

In [ ]:
# aspartate aminotrasferase is encoded by aspC (b0928) in E. coli
reaction.gene_reaction_rule = 'b0928'
reaction

In [ ]:
# gene(s) in the GPR rule are automatically added to the "genes" field of the reaction object
reaction.genes

### Add the reaction to the model

In [ ]:
# add the reaction (input as a list) to the model
model.add_reactions([reaction])

In [ ]:
# verify that the new reaction, metabolite, and gene are now in the model
model.reactions.ASPAMTR

In [ ]:
model.metabolites.asp_c

In [ ]:
model.genes.b0928.name = 'aspC'  # we can also provide the gene name
model.genes.b0928

## Flux balance analysis (FBA)

FBA is one of the most common and fundamental GEM-based analyses. It involves an optimization to estimate the flux (metabolite flow) through each reaction in the model, given the following constraints:

1. The system is at steady state - each metabolite must be consumed and produced at the same rate (represented by the equation **S**•**v** = **0**).

2. Reaction fluxes must be within their defined lower and upper bounds (`lb` and `ub`, respectively).

The optimization will seek to minimize or maximize some objective defined by the user. Most often the objective is to maximize the flux through a "biomass" reaction, which represents an organism trying to allocate its resources such that it maximizes its growth rate.

### Escher FBA

To help understand and visualize FBA and the resulting reaction fluxes, there is an excellent tool called **Escher FBA**. Follow [this link to Escher FBA](https://sbrg.github.io/escher-fba/#/), and start the browser app by clicking the Launch image.

By default, the app is maximizing flux through the `Biomass_Ecoli_core_w_GAM` reaction, and flux values are represented by reaction arrow color and thickness. Hovering over a reaction name will show some information, as well as options to knock-out the reaction or to change the objective to maximizing or minimizing flux through that reaction.

Try changing the objective to different reactions, and see how the flux distribution changes. Also try knocking out some reactions to view how it affects the results. You can reset the app using the **`Reset Map`** button in the lower right-hand corner.

### Inspect the optimization objective

Now we will perform FBA using (the less pretty but more functional) COBRApy. First let us take a look at the optimization objective (Biomass reaction by default in this model).

In [ ]:
# using cobra.util.solver:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

In [ ]:
# Alternative: use list comprehension. The "objective_coefficient" indicates
# which reactions are being maximized (1) or minimized (-1)
[x for x in model.reactions if x.objective_coefficient != 0]

In [ ]:
# view reaction details
model.reactions.Biomass_Ecoli_core

As one might expect, the biomass reaction formula is quite long and has therefore been truncated in the preview. View the full formula to see all the metabolites involved, and the (molar) ratios in which they are consumed/produced.

In [ ]:
# print entire reaction stoichiometry
model.reactions.Biomass_Ecoli_core.build_reaction_string()

In [ ]:
# view the objective direction (maximize or minimize the reaction flux)
model.objective_direction

### Perform the optimization

In [ ]:
# run FBA and view a summary of the solution object
solution = model.optimize()
solution

In [ ]:
# view a summary of the returned optimal flux distribution
model.summary()

From the summary, we can view some details beyond the value of the objective, such as which metabolites are being consumed from the media, which are being produced, and at what rate. 

In [ ]:
# get a summary of the fluxes involving a specific metabolite
model.metabolites.atp_c.summary()

In [ ]:
# get a summary of the fluxes involving a specific reaction
model.reactions.GAPD.summary()

### Change the optimization objective

In [ ]:
# let us now optimize the flux through ATPM ("ATP Maintenance"), which is just the hydrolysis of ATP
model.reactions.ATPM.build_reaction_string()

In [ ]:
# change the objective to ATPM
model.objective = 'ATPM'

This will effectively maximize the production of ATP. Can you think of why we chose the `ATPM` reaction as the objective to do this?

In [ ]:
# run FBA with the new objective
solution = model.optimize()

In [ ]:
# summarize the results
model.summary()

In [ ]:
# note that there is now no metabolic flux through the biomass reaction
#model.reactions.Biomass_Ecoli_core.summary()  # gives an error because zero flux
solution.fluxes.Biomass_Ecoli_core

In [ ]:
# set the objective back to Biomass again, for the next sections
model.objective = 'Biomass_Ecoli_core'

## Perform an _in silico_ knockout

As you may have seen in the Escher FBA app, we can simulate the effect of a gene knockout. In practice, this entails setting the upper and lower flux bounds of the associated reaction equal to zero, so that it cannot be used.

### Reaction knockout

Although in reality we knockout a **gene** from an organism, not a **reaction**, we will start with knocking out a reaction.

In [ ]:
# copy the model so we don't alter the original
model_ko = model.copy()

In [ ]:
# first optimize biomass production to view the initial maximum flux value
biomass_original = model_ko.optimize().objective_value
biomass_original  # view starting biomass flux

In [ ]:
# knock out the AKGDH reaction
model_ko.reactions.AKGDH.knock_out()
model_ko.reactions.AKGDH  # note that the upper and lower bound are now both zero

Note that the reaction is still present in the model, but it now cannot carry any flux. If we wanted to completely remove it from the model altogether, we could use the `remove_from_model` function: `model.reactions.AKGDH.remove_from_model()`

In [ ]:
# now re-optimize biomass flux
model_ko.optimize().objective_value

In [ ]:
# calculate the difference in biomass flux
biomass_original - model_ko.optimize().objective_value

Since we observe a decrease in the biomass flux after knocking out the AKGDH reaction, it indicates that the model found an alternative but less efficient pathway to generate the required biomass precursors. However, the biomass is not zero, so we would predict that this knockout would likely not be lethal to the organism.

### Gene knockout

In reality, genes are knocked out, not reactions. Let us now try knocking out the _gapA_ (b1779) gene encoding the GAPD (Glyceraldehyde-3-phosphate dehydrogenase) reaction.

In [ ]:
# copy the original model again
model_ko = model.copy()

In [ ]:
# knock out the b1779 gene
model_ko.genes.b1779.knock_out()

# check which reactions became inactive (lower bound == upper bound == 0)
[rxn.id for rxn in model_ko.reactions if rxn.upper_bound == rxn.lower_bound]

In [ ]:
# view the GAPD reaction (bounds are now zero)
model_ko.reactions.GAPD

In [ ]:
# re-optimize the knockout model
model_ko.optimize().objective_value

Here we can see that the _gapA_ gene (and its encoded GAPD reaction) were quite important, as the biomass flux has effectively been reduced to zero. This is consistent with reports that _E. coli_ [cannot grow without this gene](https://biocyc.org/gene?orgid=ECOLI&id=EG10367#tab=ESS).

### Perform all single gene or reaction deletions

Since it is a common analysis, COBRApy has specific functions for iterating through each gene (or reaction) in the model, knocking it out, and calculating the associated objective value.

In [ ]:
from cobra.flux_analysis import single_gene_deletion, single_reaction_deletion

In [ ]:
# delete all genes, one by one, and view the results
gene_del_results = single_gene_deletion(model)
gene_del_results

In [ ]:
# delete all reactions, one by one, and view the results
rxn_del_results = single_reaction_deletion(model)
rxn_del_results

### Isozyme and enzyme complex knockouts

If you still have time, try knocking out genes that encode an isozyme or a complex subunit to see what effect it has on a reaction (remember that the bounds will change to zero once it has been knocked out)

In [ ]:
# copy the original model again
model_ko = model.copy()

First try to inactivate the AKGDH reaction by knocking out one or more of its associated genes

In [ ]:
model_ko.reactions.AKGDH

Next try knocking out one or more of the genes associated with the ACALD reaction

In [ ]:
model_ko.reactions.ACALD

Did you notice anything different in how each of these reactions responds to having one of its genes knocked out? Is that consistent with your understanding of the difference between isozymes and enzyme complex subunits?